In [1]:
import os
import csv
import pickle
import random
import math
import dicom
import numpy as np
import scipy
from tqdm import tqdm
from natsort import natsorted
from skimage import transform
from sklearn.externals import joblib

import matplotlib.pyplot as plt
%matplotlib inline

#### load and shuffle data

In [2]:
(pts_train, pts_train_val, 
 data_train, label_sys_train, label_dia_train, 
 data_train_val, label_sys_train_val, label_dia_train_val, 
 data_val, data_val_pt_index) = joblib.load('../data_proc/2-data_processed.pkl')

In [3]:
shuffle_index = list(range(data_train.shape[0]))
random.shuffle(shuffle_index)
data_train = data_train[shuffle_index]
label_sys_train = label_sys_train[shuffle_index]
label_dia_train = label_dia_train[shuffle_index]

shuffle_index = list(range(data_train_val.shape[0]))
random.shuffle(shuffle_index)
data_train_val = data_train_val[shuffle_index]
label_sys_train_val = label_sys_train_val[shuffle_index]
label_dia_train_val = label_dia_train_val[shuffle_index]

#### training

In [4]:
from keras.models import Sequential, Graph
from keras.layers.core import Activation, Dense, Dropout, Flatten, Merge, Reshape, Lambda
from keras.layers.core import TimeDistributedDense, TimeDistributedMerge
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU, PReLU, ParametricSoftplus, ELU
from keras.layers.normalization import BatchNormalization
from keras.layers.noise import GaussianDropout, GaussianNoise
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using Theano backend.


Using gpu device 3: Tesla K80 (CNMeM is disabled)


Continuous Ranked Probability Score (used as loss function)

In [5]:
from keras import backend as K
from theano import tensor as T

# currently for theano, would need modifications for tensorflow

def CRPS(y_true, y_pred):
    return K.mean(K.square(T.cumsum(y_pred, axis=-1) - y_true), axis=-1)

In [6]:
model_sys = Sequential()
model_sys.add(Convolution2D(16, 7, 7, init='he_uniform', border_mode='same', dim_ordering='th', input_shape=(30, 250, 250)))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_sys.add(BatchNormalization())
model_sys.add(ELU())
model_sys.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_sys.add(Flatten())
model_sys.add(Dense(1024, activation='relu'))
model_sys.add(Dropout(0.5))
model_sys.add(Dense(600))
model_sys.add(Activation('softmax'))

model_sys.compile(loss=CRPS, optimizer='adam')

In [7]:
batch_size = 32
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/2-2-convnet_basic_systole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_sys.fit(data_train, label_sys_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=1,
              validation_data=(data_train_val, label_sys_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

Train on 5113 samples, validate on 259 samples
Epoch 1/100
5113/5113 [==============================] - 111s - loss: 0.0479 - val_loss: 0.0301
Epoch 00000: val_loss improved from inf to 0.03014, saving model to ../model_weights/2-2-convnet_basic_systole.hdf5
Epoch 2/100
5113/5113 [==============================] - 111s - loss: 0.0302 - val_loss: 0.0301
Epoch 00001: val_loss improved from 0.03014 to 0.03007, saving model to ../model_weights/2-2-convnet_basic_systole.hdf5
Epoch 3/100
5113/5113 [==============================] - 111s - loss: 0.0238 - val_loss: 0.0258
Epoch 00002: val_loss improved from 0.03007 to 0.02583, saving model to ../model_weights/2-2-convnet_basic_systole.hdf5
Epoch 4/100
5113/5113 [==============================] - 111s - loss: 0.0191 - val_loss: 0.0265
Epoch 00003: val_loss did not improve
Epoch 5/100
5113/5113 [==============================] - 111s - loss: 0.0155 - val_loss: 0.0263
Epoch 00004: val_loss did not improve
Epoch 6/100
5113/5113 [==================

In [20]:
model_dia = Sequential()
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th', input_shape=(30, 250, 250)))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(GaussianNoise(0.01))
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(GaussianNoise(0.01))
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(Convolution2D(16, 3, 3, init='he_uniform', border_mode='same', dim_ordering='th'))
model_dia.add(BatchNormalization())
model_dia.add(ELU())
model_dia.add(MaxPooling2D(pool_size=(2, 2), strides=None, border_mode='valid', dim_ordering='th'))
model_dia.add(GaussianNoise(0.01))
model_dia.add(Flatten())
model_dia.add(Dense(1024, activation='relu'))
model_dia.add(Dropout(0.2))
model_dia.add(Dense(600))
model_dia.add(Activation('softmax'))

model_dia.compile(loss=CRPS, optimizer='adam')

In [21]:
batch_size = 32
nb_epoch = 100
    
checkpointer = ModelCheckpoint(filepath='../model_weights/2-2-convnet_basic_diastole.hdf5', verbose=1, save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

model_dia.fit(data_train, label_dia_train, 
              batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=False, verbose=1,
              validation_data=(data_train_val, label_dia_train_val), shuffle=True,
              callbacks=[checkpointer, earlystopping])

Train on 5113 samples, validate on 259 samples
Epoch 1/100
5113/5113 [==============================] - 93s - loss: 0.0596 - val_loss: 0.0463
Epoch 00000: val_loss improved from inf to 0.04634, saving model to ../model_weights/2-2-convnet_basic_diastole.hdf5
Epoch 2/100
5113/5113 [==============================] - 92s - loss: 0.0415 - val_loss: 0.0458
Epoch 00001: val_loss improved from 0.04634 to 0.04584, saving model to ../model_weights/2-2-convnet_basic_diastole.hdf5
Epoch 3/100
5113/5113 [==============================] - 93s - loss: 0.0321 - val_loss: 0.0468
Epoch 00002: val_loss did not improve
Epoch 4/100
5113/5113 [==============================] - 95s - loss: 0.0243 - val_loss: 0.0500
Epoch 00003: val_loss did not improve
Epoch 5/100
 256/5113 [>.............................] - ETA: 89s - loss: 0.0193

KeyboardInterrupt: 

#### predict

In [11]:
model_sys.load_weights('../model_weights/2-0-convnet_dynwin_systole.hdf5')
model_dia.load_weights('../model_weights/2-0-convnet_dynwin_diastole.hdf5')

preds_sys = model_sys.predict(data_val, verbose=0)
preds_dia = model_dia.predict(data_val, verbose=0)

preds_sys = np.clip(np.cumsum(preds_sys, axis=-1), 0, 1)
preds_dia = np.clip(np.cumsum(preds_dia, axis=-1), 0, 1)

In [12]:
preds_sys_pt = {}
preds_dia_pt = {}
for pt in range(501, 701):
    preds_sys_pt[pt] = np.mean(preds_sys[np.where(data_val_pt_index == pt)[0]], axis=0)
    preds_dia_pt[pt] = np.mean(preds_dia[np.where(data_val_pt_index == pt)[0]], axis=0)

In [13]:
with open('../data/sample_submission_validate.csv', 'r') as fi:
    reader = csv.reader(fi)
    header = next(reader)
    
    with open('../submissions/2-0-convnet_dynwin.csv', 'w') as fo:
        writer = csv.writer(fo, lineterminator='\n')
        writer.writerow(header)
        for rowin in tqdm(reader):
            _id = rowin[0]
            pt, mode = _id.split('_')
            rowout = [_id]
            if mode.lower() == 'systole':
                rowout.extend(preds_sys_pt[int(pt)].tolist())
            elif mode.lower() == 'diastole':
                rowout.extend(preds_dia_pt[int(pt)].tolist())
            else:
                raise
            writer.writerow(rowout)